In [2]:
# import
import os
import sys
#global basepath
#print(os.path.dirname(sys.argv[0]))
#basepath = os.path.dirname(sys.argv[0]).split(__file__)[0]
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd
import numpy as np


In [3]:
# Creation of a Concrete Model
model = ConcreteModel()





#------------------- INPUTS

## Define sets ##
#  Sets
#       j        crops /sorghum, sc_adsali, sc_preseason, sc_suru/
#       m     months /m1_1 m2_1 m3_1 m4_1 m5_1 m6_1 m7_1 m8_1 m9_1 m10_1 m11_1 m12_1
#                     m1_2 m2_2 m3_2 m4_2 m5_2 m6_2 m7_2 m8_2 m9_2 m10_2 m11_2 m12_2/
# d        subdistricts /1 /

model.j = Set(initialize=['sorghum','sc_adsali', 'sc_preseason','sc_suru' ], doc='Crops')
model.m = Set(initialize=['m1_1', 'm2_1', 'm3_1', 'm4_1', 'm5_1', 'm6_1', 'm7_1', 'm8_1', 'm9_1', 'm10_1', 'm11_1', 'm12_1', 'm1_2', 'm2_2', 'm3_2', 'm4_2', 'm5_2', 'm6_2', 'm7_2', 'm8_2', 'm9_2', 'm10_2', 'm11_2', 'm12_2'], doc='Months')
model.d = Set(initialize=['1'], doc='District')						  

## Define parameters ##
#   Parameters
# land_avai_d_p(d)         Land availability by subdistrict (ha)     
# gw_table_depth_d_p(d)    Initial groundwater table depth by district (m)

model.land_avai_d_p = Param(model.d, initialize={'1':7302.3597}, doc='Land availability by subdistrict (ha)')
model.gw_table_depth_d_p = Param(model.d, initialize={'1':10}, doc='Initial groundwater table depth by district (m)')


#obs_yield_j_d_p(j,d)                                                  Observed Yields ton per ha from enterprise budgets (ton per ha)
obs_yield_tab = {
    ('sorghum',      '1') : 1.5,
    ('sc_adsali',    '1') : 120,
    ('sc_preseason', '1') : 90,
    ('sc_suru',      '1') : 70,
    }
model.obs_yield_j_d_p = Param(model.j, model.d, initialize=obs_yield_tab, doc='Observed Yields ton per ha from enterprise budgets (ton per ha)')


#sw_avai_m_d_p(d,m)                                                    Surface water availability by month and subdistrict (m3 per month)
sw_avai_tab = {
    ('1', 'm1_1')  : 150000000,
    ('1', 'm2_1')  : 150000000,
    ('1', 'm3_1')  : 150000000,
    ('1', 'm4_1')  : 150000000,
	('1', 'm5_1')  : 150000000,
    ('1', 'm6_1')  : 150000000,
    ('1', 'm7_1')  : 150000000,
    ('1', 'm8_1')  : 150000000,
	('1', 'm9_1')  : 150000000,
    ('1', 'm10_1') : 150000000,
    ('1', 'm11_1') : 150000000,
    ('1', 'm12_1') : 150000000,
	('1', 'm1_2')  : 150000000,
	('1', 'm2_2')  : 150000000,
    ('1', 'm3_2')  : 150000000,
    ('1', 'm4_2')  : 150000000,
    ('1', 'm5_2')  : 150000000,
	('1', 'm6_2')  : 150000000,
    ('1', 'm7_2')  : 150000000,
    ('1', 'm8_2')  : 150000000,
    ('1', 'm9_2')  : 150000000,
	('1', 'm10_2') : 150000000,
    ('1', 'm11_2') : 150000000,
    ('1', 'm12_2') : 150000000,
    }
model.sw_avai_m_d_p = Param(model.d, model.m, initialize=sw_avai_tab, doc='Surface water availability by month and subdistrict (m3 per month)')

#gw_avai_m_d_p(d,m)                                                    Groundwater availability by month and subdistrict (m3 per month)
gw_avai_tab = {
    ('1', 'm1_1')  : 150000000,
    ('1', 'm2_1')  : 150000000,
    ('1', 'm3_1')  : 150000000,
    ('1', 'm4_1')  : 150000000,
	('1', 'm5_1')  : 150000000,
    ('1', 'm6_1')  : 150000000,
    ('1', 'm7_1')  : 150000000,
    ('1', 'm8_1')  : 150000000,
	('1', 'm9_1')  : 150000000,
    ('1', 'm10_1') : 150000000,
    ('1', 'm11_1') : 150000000,
    ('1', 'm12_1') : 150000000,
	('1', 'm1_2')  : 150000000,
	('1', 'm2_2')  : 150000000,
    ('1', 'm3_2')  : 150000000,
    ('1', 'm4_2')  : 150000000,
    ('1', 'm5_2')  : 150000000,
	('1', 'm6_2')  : 150000000,
    ('1', 'm7_2')  : 150000000,
    ('1', 'm8_2')  : 150000000,
    ('1', 'm9_2')  : 150000000,
	('1', 'm10_2') : 150000000,
    ('1', 'm11_2') : 150000000,
    ('1', 'm12_2') : 150000000,
    }
model.gw_avai_m_d_p = Param(model.d, model.m, initialize=gw_avai_tab, doc='Groundwater availability by month and subdistrict (m3 per month)')
 
#crop_price_j_d_p(j,d)                                                 Crop prices from enterprise budgets (USD per ton)
crop_price_tab = {
    ('sorghum',      '1') : 150,
    ('sc_adsali',    '1') : 20,
    ('sc_preseason', '1') : 20,
    ('sc_suru',      '1') : 20,
    }
model.crop_price_j_d_p = Param(model.j, model.d, initialize=crop_price_tab, doc='Crop prices from enterprise budgets (USD per ton)')

#linear_land_cost_j_d_p(j,d)                                           Linear land production cost (USD per ha)
linear_land_cost_tab = {
    ('sorghum',      '1') : 153,
    ('sc_adsali',    '1') : 1632,
    ('sc_preseason', '1') : 1224,
    ('sc_suru','1') : 952,
    }
model.linear_land_cost_j_d_p = Param(model.j, model.d, initialize=linear_land_cost_tab, doc='Linear land production cost (USD per ha)')

#sw_cost_j_d_p(j,d)                                                    Surface water cost (USD per m3)  
sw_cost_tab = {
    ('sorghum',      '1') : 0,
    ('sc_adsali',    '1') : 0,
    ('sc_preseason', '1') : 0,
    ('sc_suru',      '1') : 0,
    }
model.sw_cost_j_d_p = Param(model.j, model.d, initialize=sw_cost_tab, doc='Surface water cost (USD per m3)')

#gw_cost_j_d_p(j,d)                                                    groundwater water cost (USD per m per m3)  
gw_cost_tab = {
    ('sorghum',      '1') : 0,
    ('sc_adsali',    '1') : 0,
    ('sc_preseason', '1') : 0,
    ('sc_suru',      '1') : 0,
    }
model.gw_cost_j_d_p = Param(model.j, model.d, initialize=gw_cost_tab, doc='groundwater water cost (USD per m per m3)')

#alpha_j_d_p(j,d)                                                      PMP alpha parameter 
alpha_tab = {
    ('sorghum',      '1') : 0,
    ('sc_adsali',    '1') : -696,
    ('sc_preseason', '1') : -504,
    ('sc_suru',      '1') : -376,
    }
model.alpha_j_d_p = Param(model.j, model.d, initialize=alpha_tab, doc='PMP alpha parameter')

#gamma_j_d_p(j,d)                                                      PMP gamma parameter
gamma_tab = {
    ('sorghum',      '1') : 0,
    ('sc_adsali',    '1') : 1.88150670840944,
    ('sc_preseason', '1') : 0.491529215657858,
    ('sc_suru',      '1') : 0.962753105166822,
    }
model.gamma_j_d_p = Param(model.j, model.d, initialize=gamma_tab, doc='PMP gamma parameter')

#grs_ir_wat_req_j_m_d_p(m,j,d)                                         Monthly gross irrigation water requirements by crop (m3 per month per ha)
grs_ir_wat_req_tab = { 
   ('m1_1','sc_suru','1'):	312.0167153,
   ('m2_1','sc_suru','1'):	1717.859248,
   ('m3_1','sc_suru','1'):	2248.724687,
   ('m4_1','sc_suru','1'):	3941.972648,
   ('m5_1','sc_suru','1'):	3597.874777,
   ('m6_1','sc_suru','1'):	649.83121,
   ('m6_1','sc_adsali','1'):	2753.809563,
   ('m7_1','sc_adsali','1'):	626.9628687,
   ('m7_1','sc_suru','1'):	872.8099213,
   ('m8_1','sc_adsali','1'):	749.7847653,
   ('m8_1','sc_suru','1'):	1390.920069,
   ('m9_1','sc_adsali','1'):	1493.00432,
   ('m9_1','sc_suru','1'):	2332.81925,
   ('m10_1','sorghum','1'):	1429.92584,
   ('m10_1','sc_adsali','1'):	1634.20096,
   ('m10_1','sc_preseason','1'):	817.10048,
   ('m10_1','sc_suru','1'):	306.1828667,
   ('m11_1','sorghum','1'):	1539.303455,
   ('m11_1','sc_adsali','1'):	2109.057835,
   ('m11_1','sc_preseason','1'):	1320.167155,
   ('m11_1','sc_suru','1'):	1232.512635,
   ('m12_1','sorghum','1'):	2015.057428,
   ('m12_1','sc_adsali','1'):	2190.279813,
   ('m12_1','sc_preseason','1'):	1401.77908,
   ('m12_1','sc_suru','1'):	1314.167888,
   ('m1_2','sorghum','1'):	1023.566378,
   ('m1_2','sc_adsali','1'):	2371.671174,
   ('m1_2','sc_preseason','1'):	1505.032377,
   ('m2_2','sc_adsali','1'):	2800.224486,
   ('m2_2','sc_preseason','1'):	2800.224486,
   ('m3_2','sc_adsali','1'):	3860.066048,
   ('m3_2','sc_preseason','1'):	3860.066048,
   ('m4_2','sc_adsali','1'):	4183.612985,
   ('m4_2','sc_preseason','1'):	4183.612985,
   ('m5_2','sc_adsali','1'):	4628.341135,
   ('m5_2','sc_preseason','1'):	4628.341135,
   ('m6_2','sc_adsali','1'):	988.734375,
   ('m6_2','sc_preseason','1'):	988.734375,
   ('m7_2','sc_adsali','1'):	1820.243921,
   ('m7_2','sc_preseason','1'):	1820.243921,
   ('m8_2','sc_adsali','1'):	602.5430992,
   ('m8_2','sc_preseason','1'):	1221.836454,
   ('m9_2','sc_adsali','1'):	1360.01565,
   ('m9_2','sc_preseason','1'):	136.68325,
   ('m10_2','sc_adsali','1'):	1545.703788,
   ('m10_2','sc_preseason','1'):	1545.703788,
   ('m11_2','sc_preseason','1'):	1256.444275,
   ('m12_2','sc_preseason','1'):	1441.864095,  
   ('m1_1'	,'sorghum','1'):0,
   ('m1_1'	,'sc_adsali','1'):0,
   ('m1_1'	,'sc_preseason','1'):0,
   ('m2_1'	,'sorghum','1'):0,
   ('m2_1'	,'sc_adsali','1'):0,
   ('m2_1'	,'sc_preseason','1'):0,
   ('m3_1'	,'sorghum','1'):0,
   ('m3_1'	,'sc_adsali','1'):0,
   ('m3_1'	,'sc_preseason','1'):0,
   ('m4_1'	,'sorghum','1'):0,
   ('m4_1'	,'sc_adsali','1'):0,
   ('m4_1'	,'sc_preseason','1'):0,
   ('m5_1'	,'sorghum','1'):0,
   ('m5_1'	,'sc_adsali','1'):0,
   ('m5_1'	,'sc_preseason','1'):0,
   ('m6_1'	,'sorghum','1'):0,
   ('m6_1'	,'sc_preseason','1'):0,
   ('m7_1'	,'sorghum','1'):0,
   ('m7_1'	,'sc_preseason','1'):0,
   ('m8_1'	,'sorghum','1'):0,
   ('m8_1'	,'sc_preseason','1'):0,
   ('m9_1'	,'sorghum','1'):0,
   ('m9_1'	,'sc_preseason','1'):0,
   ('m1_2'	,'sc_suru','1'):0,
   ('m2_2'	,'sorghum','1'):0,
   ('m2_2'	,'sc_suru','1'):0,
   ('m3_2'	,'sorghum','1'):0,
   ('m3_2'	,'sc_suru','1'):0,
   ('m4_2'	,'sorghum','1'):0,
   ('m4_2'	,'sc_suru','1'):0,
   ('m5_2'	,'sorghum','1'):0,
   ('m5_2'	,'sc_suru','1'):0,
   ('m6_2'	,'sorghum','1'):0,
   ('m6_2'	,'sc_suru','1'):0,
   ('m7_2'	,'sorghum','1'):0,
   ('m7_2'	,'sc_suru','1'):0,
   ('m8_2'	,'sorghum','1'):0,
   ('m8_2'	,'sc_suru','1'):0,
   ('m9_2'	,'sorghum','1'):0,
   ('m9_2'	,'sc_suru','1'):0,
   ('m10_2'	,'sorghum','1'):0,
   ('m10_2'	,'sc_suru','1'):0,
   ('m11_2'	,'sorghum','1'):0,
   ('m11_2'	,'sc_adsali','1'):0,
   ('m11_2','sc_suru','1'):0,
   ('m12_2'	,'sorghum','1'):0,
   ('m12_2'	,'sc_adsali','1'):0,
   ('m12_2'	,'sc_suru','1'):0, 
   }
model.grs_ir_wat_req_j_m_d_p = Param(model.m, model.j, model.d, initialize=grs_ir_wat_req_tab, doc='Monthly gross irrigation water requirements by crop (m3 per month per ha)')




#-------------------------- VARS


## Define variables ##

#Variables
#tot_ir_wat_use_opt_j_m_d_v         (m,j,d)                                      Irrigation water use per month and district (m3)
model.tot_ir_wat_use_opt_j_m_d_v = Var(model.m,model.j, model.d, bounds=(None,None), doc='Irrigation water use per month and district (m3)')

#revenue_opt_j_d_v                  (  j,d)                                      Crop revenue per crop and district (USD)
model.revenue_opt_j_d_v = Var(model.j,model.d, bounds=(None,None), doc='Crop revenue per crop and district (USD)')

#prod_cost_opt_j_d_v                (  j,d)                                      Production costs excluding water costs per crop and district (USD)
model.prod_cost_opt_j_d_v = Var(model.j,model.d, bounds=(None,None), doc='Production costs excluding water costs per crop and district (USD)')

#sw_cost_opt_j_d_v                  (  j,d)                                      SW use cost per crop and district (USD)
model.sw_cost_opt_j_d_v = Var(model.j,model.d, bounds=(None,None), doc='SW use cost per crop and district (USD)')

#income_opt_j_d_v                   (j,  d)                                      Farm income by crop and irrigation subdistrict (USD)
model.income_opt_j_d_v = Var(model.j,model.d, bounds=(None,None), doc='Farm income by crop and irrigation subdistrict (USD)')

#income_opt_d_v                     (    d)                                      Total farm income in each time step (USD)
model.income_opt_d_v = Var(model.d, bounds=(None,None), doc='Total farm income in each time step (USD)')

#Positive Variables

#yr_sw_use_opt_j_d_v                (  j,d)
model.yr_sw_use_opt_j_d_v= Var(model.j, model.d, bounds=(0.0,None), doc='Yearly surface water use per crop and district (m3)')

#land_opt_j_d_v                     (j,  d)                                      Optimal land allocation (ha)
model.land_opt_j_d_v = Var(model.j, model.d, bounds=(0.0,None), doc='Optimal land allocation (ha)')




### --------------- CONSTRAINTS

## Irrigation Water Requirement (water depth * land area = water volume)
def tot_ir_wat_use_opt_e(model, m,j,d):
  return model.grs_ir_wat_req_j_m_d_p[m,j,d] * model.land_opt_j_d_v[j,d] == model.tot_ir_wat_use_opt_j_m_d_v[m,j,d]
model.tot_ir_wat_use_opt_j_m_d_e = Constraint(model.m, model.j, model.d, rule=tot_ir_wat_use_opt_e, doc='Irrigation water use per month and district')


# Land availability ( land < avail land -- hardcoded for 1 district for now)
def land_constraint_opti_e(model, d):
  return sum(model.land_opt_j_d_v[j,d] for j in model.j) <= model.land_avai_d_p[d] 
model.land_constraint_opt_e = Constraint(model.d, rule=land_constraint_opti_e, doc='Land availability')


# Crop Revenue (price in $/ton *yield in ton/ha *land in ha)
def revenue_opt_e(model, j,d):
  return model.crop_price_j_d_p[j,d] * model.obs_yield_j_d_p[j,d] * model.land_opt_j_d_v[j,d] == model.revenue_opt_j_d_v[j,d] 
model.revenue_opt_j_d_e = Constraint(model.j,model.d, rule=revenue_opt_e, doc='Crop revenue per crop and district')


#Production Cost (unit cost in $/ha *ha)
def prod_cost_opt_e(model, j,d):
  return model.linear_land_cost_j_d_p[j,d] * model.land_opt_j_d_v[j,d] == model.prod_cost_opt_j_d_v[j,d] 
model.prod_cost_opt_j_d_e = Constraint(model.j,model.d, rule=prod_cost_opt_e, doc='Production costs excluding water costs per crop and district')

#Yearly surface water use (sum of monthly water use)
def yr_sw_use_opt_e(model, j,d):
  return sum(model.tot_ir_wat_use_opt_j_m_d_v[m,j,d] for m in model.m) == model.yr_sw_use_opt_j_d_v[j,d] 
model.yr_sw_use_opt_j_d_e = Constraint(model.j,model.d, rule=yr_sw_use_opt_e, doc='Yearly SW use per crop and district')


#sw_cost_opt_j_d_e                  (  j,d)                                      SW use cost per crop and district
#sw_cost_opt_j_d_e                  (  j,d)..     sw_cost_opt_j_d_v              (  j,d)  =e= [sw_cost_j_d_p(j,d)             * sum(m, sw_use_opt_j_m_d_v(m,j,d)) ] ;
def sw_cost_opt_e(model, j,d):
  return model.sw_cost_j_d_p[j,d] * model.yr_sw_use_opt_j_d_v[j,d] == model.sw_cost_opt_j_d_v[j,d] 
model.sw_cost_opt_j_d_e = Constraint(model.j, model.d, rule=sw_cost_opt_e, doc='SW use cost per crop and district')

#income_opt_j_d_e                   (j,  d)                                      Net income by crop and irrigation subdistrict
#income_opt_j_d_e                   (  j,d)..     income_opt_j_d_v               (  j,d)  =e= [revenue_opt_j_d_v(j,d) - prod_cost_opt_j_d_v(j,d) - sw_cost_opt_j_d_v(j,d) - gw_cost_opt_j_d_v(j,d)]
#                                                                                         -   [alpha_j_d_p(j,d) * Land_opt_j_d_v(j,d)] - [0.5 * gamma_j_d_p(j,d) * Land_opt_j_d_v(j,d)**2] ;
def income_opt_e(model, j,d):
  return (model.revenue_opt_j_d_v[j,d] - model.prod_cost_opt_j_d_v[j,d] - model.sw_cost_opt_j_d_v[j,d]) - (model.alpha_j_d_p[j,d] * model.land_opt_j_d_v[j,d]) - (0.5 * model.gamma_j_d_p[j,d] * model.land_opt_j_d_v[j,d] * model.land_opt_j_d_v[j,d]) == model.income_opt_j_d_v[j,d] 
model.income_opt_j_d_e = Constraint(model.j,model.d, rule=income_opt_e, doc='Net income by crop and irrigation subdistrict')

#income_opt_d_e                     (    d)                                      Net income in ach time step
#income_opt_d_e                     (    d)..     income_opt_d_v                 (    d)  =e= sum(j, income_opt_j_d_v(j,d)) ;
def tot_income_opt_e(model, d):
  return sum(model.income_opt_j_d_v[j,d] for j in model.j) == model.income_opt_d_v[d] 
model.income_opt_d_e = Constraint(model.d, rule=tot_income_opt_e, doc='Net income in ach time step')


## Define Objective and solve ##
#tot_income_opt_e                                                                Total income over all time steps  
#tot_income_opt_e                          ..     tot_income_opt_v                        =e= sum(d, income_opt_d_v(d)) ; 
#Model Pune1 /all/;
#Solve Pune1 using nlp maximizing tot_income_opt_v; 
def objective_rule(model):
  return sum(model.income_opt_d_v[d] for d in model.d)
model.objective = Objective(rule=objective_rule, sense=maximize, doc='Total income over all time steps ')

##Display everything in your model
#model.pprint()

##Display the constraints of the model 
#for con in model.component_map(Constraint).itervalues():
#    con.pprint()

## Creating and running the solver:
opt = SolverFactory("ipopt", solver_io='nl')
opt.options['max_iter']= 1000000 #number of iterations you wish
results = opt.solve(model,keepfiles=False, tee=True)
print(results.solver.termination_condition)
results.write()


##def pyomo_postprocess(options=None, instance=None, results=None):
model.land_opt_j_d_v.display()
model.objective.display()
 




Ipopt 3.12.12: max_iter=1000000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      288
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:      121
                     variables with only lower bounds:        8
                variables with lower and upper bounds:        0
                     variables with only upper bou

In [17]:
# Creation of a Concrete Model
model = ConcreteModel()





#------------------- INPUTS

## Define sets ##
#  Sets
#       j        crops /sorghum, sc_adsali, sc_preseason, sc_suru/
#       m     months /m1_1 m2_1 m3_1 m4_1 m5_1 m6_1 m7_1 m8_1 m9_1 m10_1 m11_1 m12_1
#                     m1_2 m2_2 m3_2 m4_2 m5_2 m6_2 m7_2 m8_2 m9_2 m10_2 m11_2 m12_2/
# d        subdistricts /1 /

model.j = Set(initialize=['sorghum','sc_adsali', 'sc_preseason','sc_suru' ], doc='Crops')
model.m = Set(initialize=['m1_1', 'm2_1', 'm3_1', 'm4_1', 'm5_1', 'm6_1', 'm7_1', 'm8_1', 'm9_1', 'm10_1', 'm11_1', 'm12_1', 'm1_2', 'm2_2', 'm3_2', 'm4_2', 'm5_2', 'm6_2', 'm7_2', 'm8_2', 'm9_2', 'm10_2', 'm11_2', 'm12_2'], doc='Months')
model.d = Set(initialize=['1'], doc='District')						  

## Define parameters ##
#   Parameters
# land_avai_d_p(d)         Land availability by subdistrict (ha)     
# gw_table_depth_d_p(d)    Initial groundwater table depth by district (m)

model.land_avai_d_p = Param(model.d, initialize={'1':7302.3597}, doc='Land availability by subdistrict (ha)')
model.gw_table_depth_d_p = Param(model.d, initialize={'1':10}, doc='Initial groundwater table depth by district (m)')


#obs_yield_j_d_p(j,d)                                                  Observed Yields ton per ha from enterprise budgets (ton per ha)
obs_yield_tab = {
    ('sorghum',      '1') : 1.5,
    ('sc_adsali',    '1') : 120,
    ('sc_preseason', '1') : 90,
    ('sc_suru',      '1') : 70,
    }
model.obs_yield_j_d_p = Param(model.j, model.d, initialize=obs_yield_tab, doc='Observed Yields ton per ha from enterprise budgets (ton per ha)')


#sw_avai_m_d_p(d,m)                                                    Surface water availability by month and subdistrict (m3 per month)
sw_avai_tab = {
    ('1', 'm1_1')  : 150000000,
    ('1', 'm2_1')  : 150000000,
    ('1', 'm3_1')  : 150000000,
    ('1', 'm4_1')  : 150000000,
	('1', 'm5_1')  : 150000000,
    ('1', 'm6_1')  : 150000000,
    ('1', 'm7_1')  : 150000000,
    ('1', 'm8_1')  : 150000000,
	('1', 'm9_1')  : 150000000,
    ('1', 'm10_1') : 150000000,
    ('1', 'm11_1') : 150000000,
    ('1', 'm12_1') : 150000000,
	('1', 'm1_2')  : 150000000,
	('1', 'm2_2')  : 150000000,
    ('1', 'm3_2')  : 150000000,
    ('1', 'm4_2')  : 150000000,
    ('1', 'm5_2')  : 150000000,
	('1', 'm6_2')  : 150000000,
    ('1', 'm7_2')  : 150000000,
    ('1', 'm8_2')  : 150000000,
    ('1', 'm9_2')  : 150000000,
	('1', 'm10_2') : 150000000,
    ('1', 'm11_2') : 150000000,
    ('1', 'm12_2') : 150000000,
    }
model.sw_avai_m_d_p = Param(model.d, model.m, initialize=sw_avai_tab, doc='Surface water availability by month and subdistrict (m3 per month)')

#gw_avai_m_d_p(d,m)                                                    Groundwater availability by month and subdistrict (m3 per month)
gw_avai_tab = {
    ('1', 'm1_1')  : 150000000,
    ('1', 'm2_1')  : 150000000,
    ('1', 'm3_1')  : 150000000,
    ('1', 'm4_1')  : 150000000,
	('1', 'm5_1')  : 150000000,
    ('1', 'm6_1')  : 150000000,
    ('1', 'm7_1')  : 150000000,
    ('1', 'm8_1')  : 150000000,
	('1', 'm9_1')  : 150000000,
    ('1', 'm10_1') : 150000000,
    ('1', 'm11_1') : 150000000,
    ('1', 'm12_1') : 150000000,
	('1', 'm1_2')  : 150000000,
	('1', 'm2_2')  : 150000000,
    ('1', 'm3_2')  : 150000000,
    ('1', 'm4_2')  : 150000000,
    ('1', 'm5_2')  : 150000000,
	('1', 'm6_2')  : 150000000,
    ('1', 'm7_2')  : 150000000,
    ('1', 'm8_2')  : 150000000,
    ('1', 'm9_2')  : 150000000,
	('1', 'm10_2') : 150000000,
    ('1', 'm11_2') : 150000000,
    ('1', 'm12_2') : 150000000,
    }
model.gw_avai_m_d_p = Param(model.d, model.m, initialize=gw_avai_tab, doc='Groundwater availability by month and subdistrict (m3 per month)')
  
#crop_price_j_d_p(j,d)                                                 Crop prices from enterprise budgets (USD per ton)
crop_price_tab = {
    ('sorghum',      '1') : 150,
    ('sc_adsali',    '1') : 20,
    ('sc_preseason', '1') : 20,
    ('sc_suru',      '1') : 20,
    }
model.crop_price_j_d_p = Param(model.j, model.d, initialize=crop_price_tab, doc='Crop prices from enterprise budgets (USD per ton)')

#linear_land_cost_j_d_p(j,d)                                           Linear land production cost (USD per ha)
linear_land_cost_tab = {
    ('sorghum',      '1') : 153,
    ('sc_adsali',    '1') : 1632,
    ('sc_preseason', '1') : 1224,
    ('sc_suru','1') : 952,
    }
model.linear_land_cost_j_d_p = Param(model.j, model.d, initialize=linear_land_cost_tab, doc='Linear land production cost (USD per ha)')

#sw_cost_j_d_p(j,d)                                                    Surface water cost (USD per m3)  
sw_cost_tab = {
    ('sorghum',      '1') : 0,
    ('sc_adsali',    '1') : 0,
    ('sc_preseason', '1') : 0,
    ('sc_suru',      '1') : 0,
    }
model.sw_cost_j_d_p = Param(model.j, model.d, initialize=sw_cost_tab, doc='Surface water cost (USD per m3)')

#gw_cost_j_d_p(j,d)                                                    groundwater water cost (USD per m per m3)  
gw_cost_tab = {
    ('sorghum',      '1') : 0,
    ('sc_adsali',    '1') : 0,
    ('sc_preseason', '1') : 0,
    ('sc_suru',      '1') : 0,
    }
model.gw_cost_j_d_p = Param(model.j, model.d, initialize=gw_cost_tab, doc='groundwater water cost (USD per m per m3)')

#alpha_j_d_p(j,d)                                                      PMP alpha parameter 
alpha_tab = {
    ('sorghum',      '1') : 0,
    ('sc_adsali',    '1') : -696,
    ('sc_preseason', '1') : -504,
    ('sc_suru',      '1') : -376,
    }
model.alpha_j_d_p = Param(model.j, model.d, initialize=alpha_tab, doc='PMP alpha parameter')

#gamma_j_d_p(j,d)                                                      PMP gamma parameter
gamma_tab = {
    ('sorghum',      '1') : 0,
    ('sc_adsali',    '1') : 1.88150670840944,
    ('sc_preseason', '1') : 0.491529215657858,
    ('sc_suru',      '1') : 0.962753105166822,
    }
model.gamma_j_d_p = Param(model.j, model.d, initialize=gamma_tab, doc='PMP gamma parameter')

#grs_ir_wat_req_j_m_d_p(m,j,d)                                         Monthly gross irrigation water requirements by crop (m3 per month per ha)
grs_ir_wat_req_tab = { 
   ('m1_1','sc_suru','1'):	312.0167153,
   ('m2_1','sc_suru','1'):	1717.859248,
   ('m3_1','sc_suru','1'):	2248.724687,
   ('m4_1','sc_suru','1'):	3941.972648,
   ('m5_1','sc_suru','1'):	3597.874777,
   ('m6_1','sc_suru','1'):	649.83121,
   ('m6_1','sc_adsali','1'):	2753.809563,
   ('m7_1','sc_adsali','1'):	626.9628687,
   ('m7_1','sc_suru','1'):	872.8099213,
   ('m8_1','sc_adsali','1'):	749.7847653,
   ('m8_1','sc_suru','1'):	1390.920069,
   ('m9_1','sc_adsali','1'):	1493.00432,
   ('m9_1','sc_suru','1'):	2332.81925,
   ('m10_1','sorghum','1'):	1429.92584,
   ('m10_1','sc_adsali','1'):	1634.20096,
   ('m10_1','sc_preseason','1'):	817.10048,
   ('m10_1','sc_suru','1'):	306.1828667,
   ('m11_1','sorghum','1'):	1539.303455,
   ('m11_1','sc_adsali','1'):	2109.057835,
   ('m11_1','sc_preseason','1'):	1320.167155,
   ('m11_1','sc_suru','1'):	1232.512635,
   ('m12_1','sorghum','1'):	2015.057428,
   ('m12_1','sc_adsali','1'):	2190.279813,
   ('m12_1','sc_preseason','1'):	1401.77908,
   ('m12_1','sc_suru','1'):	1314.167888,
   ('m1_2','sorghum','1'):	1023.566378,
   ('m1_2','sc_adsali','1'):	2371.671174,
   ('m1_2','sc_preseason','1'):	1505.032377,
   ('m2_2','sc_adsali','1'):	2800.224486,
   ('m2_2','sc_preseason','1'):	2800.224486,
   ('m3_2','sc_adsali','1'):	3860.066048,
   ('m3_2','sc_preseason','1'):	3860.066048,
   ('m4_2','sc_adsali','1'):	4183.612985,
   ('m4_2','sc_preseason','1'):	4183.612985,
   ('m5_2','sc_adsali','1'):	4628.341135,
   ('m5_2','sc_preseason','1'):	4628.341135,
   ('m6_2','sc_adsali','1'):	988.734375,
   ('m6_2','sc_preseason','1'):	988.734375,
   ('m7_2','sc_adsali','1'):	1820.243921,
   ('m7_2','sc_preseason','1'):	1820.243921,
   ('m8_2','sc_adsali','1'):	602.5430992,
   ('m8_2','sc_preseason','1'):	1221.836454,
   ('m9_2','sc_adsali','1'):	1360.01565,
   ('m9_2','sc_preseason','1'):	136.68325,
   ('m10_2','sc_adsali','1'):	1545.703788,
   ('m10_2','sc_preseason','1'):	1545.703788,
   ('m11_2','sc_preseason','1'):	1256.444275,
   ('m12_2','sc_preseason','1'):	1441.864095,  
   ('m1_1'	,'sorghum','1'):0,
   ('m1_1'	,'sc_adsali','1'):0,
   ('m1_1'	,'sc_preseason','1'):0,
   ('m2_1'	,'sorghum','1'):0,
   ('m2_1'	,'sc_adsali','1'):0,
   ('m2_1'	,'sc_preseason','1'):0,
   ('m3_1'	,'sorghum','1'):0,
   ('m3_1'	,'sc_adsali','1'):0,
   ('m3_1'	,'sc_preseason','1'):0,
   ('m4_1'	,'sorghum','1'):0,
   ('m4_1'	,'sc_adsali','1'):0,
   ('m4_1'	,'sc_preseason','1'):0,
   ('m5_1'	,'sorghum','1'):0,
   ('m5_1'	,'sc_adsali','1'):0,
   ('m5_1'	,'sc_preseason','1'):0,
   ('m6_1'	,'sorghum','1'):0,
   ('m6_1'	,'sc_preseason','1'):0,
   ('m7_1'	,'sorghum','1'):0,
   ('m7_1'	,'sc_preseason','1'):0,
   ('m8_1'	,'sorghum','1'):0,
   ('m8_1'	,'sc_preseason','1'):0,
   ('m9_1'	,'sorghum','1'):0,
   ('m9_1'	,'sc_preseason','1'):0,
   ('m1_2'	,'sc_suru','1'):0,
   ('m2_2'	,'sorghum','1'):0,
   ('m2_2'	,'sc_suru','1'):0,
   ('m3_2'	,'sorghum','1'):0,
   ('m3_2'	,'sc_suru','1'):0,
   ('m4_2'	,'sorghum','1'):0,
   ('m4_2'	,'sc_suru','1'):0,
   ('m5_2'	,'sorghum','1'):0,
   ('m5_2'	,'sc_suru','1'):0,
   ('m6_2'	,'sorghum','1'):0,
   ('m6_2'	,'sc_suru','1'):0,
   ('m7_2'	,'sorghum','1'):0,
   ('m7_2'	,'sc_suru','1'):0,
   ('m8_2'	,'sorghum','1'):0,
   ('m8_2'	,'sc_suru','1'):0,
   ('m9_2'	,'sorghum','1'):0,
   ('m9_2'	,'sc_suru','1'):0,
   ('m10_2'	,'sorghum','1'):0,
   ('m10_2'	,'sc_suru','1'):0,
   ('m11_2'	,'sorghum','1'):0,
   ('m11_2'	,'sc_adsali','1'):0,
   ('m11_2','sc_suru','1'):0,
   ('m12_2'	,'sorghum','1'):0,
   ('m12_2'	,'sc_adsali','1'):0,
   ('m12_2'	,'sc_suru','1'):0, 
   }
model.grs_ir_wat_req_j_m_d_p = Param(model.m, model.j, model.d, initialize=grs_ir_wat_req_tab, doc='Monthly gross irrigation water requirements by crop (m3 per month per ha)')




#-------------------------- VARS


## Define variables ##

#Variables
#tot_ir_wat_use_opt_j_m_d_v         (m,j,d)                                      Irrigation water use per month and district (m3)
model.tot_ir_wat_use_opt_j_m_d_v = Var(model.m,model.j, model.d, bounds=(None,None), doc='Irrigation water use per month and district (m3)')

#revenue_opt_j_d_v                  (  j,d)                                      Crop revenue per crop and district (USD)
model.revenue_opt_j_d_v = Var(model.j,model.d, bounds=(None,None), doc='Crop revenue per crop and district (USD)')

#prod_cost_opt_j_d_v                (  j,d)                                      Production costs excluding water costs per crop and district (USD)
model.prod_cost_opt_j_d_v = Var(model.j,model.d, bounds=(None,None), doc='Production costs excluding water costs per crop and district (USD)')

#sw_cost_opt_j_d_v                  (  j,d)                                      SW use cost per crop and district (USD)
model.sw_cost_opt_j_d_v = Var(model.j,model.d, bounds=(None,None), doc='SW use cost per crop and district (USD)')

#income_opt_j_d_v                   (j,  d)                                      Farm income by crop and irrigation subdistrict (USD)
model.income_opt_j_d_v = Var(model.j,model.d, bounds=(None,None), doc='Farm income by crop and irrigation subdistrict (USD)')

#income_opt_d_v                     (    d)                                      Total farm income in each time step (USD)
model.income_opt_d_v = Var(model.d, bounds=(None,None), doc='Total farm income in each time step (USD)')

#Positive Variables

#yr_sw_use_opt_j_d_v                (  j,d)
model.yr_sw_use_opt_j_d_v= Var(model.j, model.d, bounds=(0.0,None), doc='Yearly surface water use per crop and district (m3)')

#land_opt_j_d_v                     (j,  d)                                      Optimal land allocation (ha)
model.land_opt_j_d_v = Var(model.j, model.d, bounds=(0.0,None), doc='Optimal land allocation (ha)')

#gw_depth_opt_d_v
model.gw_depth_opt_d_v = Var(model.d,bounds=(0,None),doc='GW depth (m)??')



### --------------- CONSTRAINTS

## Irrigation Water Requirement (water depth * land area = water volume)
def tot_ir_wat_use_opt_e(model, m,j,d):
  return model.grs_ir_wat_req_j_m_d_p[m,j,d] * model.land_opt_j_d_v[j,d] == model.tot_ir_wat_use_opt_j_m_d_v[m,j,d]
model.tot_ir_wat_use_opt_j_m_d_e = Constraint(model.m, model.j, model.d, rule=tot_ir_wat_use_opt_e, doc='Irrigation water use per month and district')


# Land availability ( land < avail land -- hardcoded for 1 district for now)
def land_constraint_opti_e(model, d):
  return sum(model.land_opt_j_d_v[j,d] for j in model.j) <= model.land_avai_d_p[d] 
model.land_constraint_opt_e = Constraint(model.d, rule=land_constraint_opti_e, doc='Land availability')


# Crop Revenue (price in $/ton *yield in ton/ha *land in ha)
def revenue_opt_e(model, j,d):
  return model.crop_price_j_d_p[j,d] * model.obs_yield_j_d_p[j,d] * model.land_opt_j_d_v[j,d] == model.revenue_opt_j_d_v[j,d] 
model.revenue_opt_j_d_e = Constraint(model.j,model.d, rule=revenue_opt_e, doc='Crop revenue per crop and district')

#Production Cost (unit cost in $/ha *ha)
def prod_cost_opt_e(model, j,d):
  return model.linear_land_cost_j_d_p[j,d] * model.land_opt_j_d_v[j,d] == model.prod_cost_opt_j_d_v[j,d] 
model.prod_cost_opt_j_d_e = Constraint(model.j,model.d, rule=prod_cost_opt_e, doc='Production costs excluding water costs per crop and district')

#Yearly surface water use (sum of monthly water use)
def yr_sw_use_opt_e(model, j,d):
  return sum(model.tot_ir_wat_use_opt_j_m_d_v[m,j,d] for m in model.m) == model.yr_sw_use_opt_j_d_v[j,d] 
model.yr_sw_use_opt_j_d_e = Constraint(model.j,model.d, rule=yr_sw_use_opt_e, doc='Yearly SW use per crop and district')

#sw_cost_opt_j_d_e                  (  j,d)                                      SW use cost per crop and district
#sw_cost_opt_j_d_e                  (  j,d)..     sw_cost_opt_j_d_v              (  j,d)  =e= [sw_cost_j_d_p(j,d)             * sum(m, sw_use_opt_j_m_d_v(m,j,d)) ] ;
def sw_cost_opt_e(model, j,d):
  return model.sw_cost_j_d_p[j,d] * model.yr_sw_use_opt_j_d_v[j,d] == model.sw_cost_opt_j_d_v[j,d] 
model.sw_cost_opt_j_d_e = Constraint(model.j, model.d, rule=sw_cost_opt_e, doc='SW use cost per crop and district')

#gw_cost_opt_j_d_e                  (  j,d)..     gw_cost_opt_j_d_v              (  j,d)  =e= [gw_cost_j_d_p(j,d) * gw_depth_opt_d_v(d) * sum(m, gw_use_opt_j_m_d_v(m,j,d)) ] ;
def gw_cost_opt_e(model, j,d):
  return model.gw_cost_j_d_p[j,d] * model.gw_depth_opt_d_v[d] * sum(gw_use_opt_j_m_d_v[m,j,d] for m in model.m) == model.gw_cost_opt_j_d_v[j,d] 
model.gw_cost_opt_j_d_e = Constraint(model.j, model.d, rule=gw_cost_opt_e, doc='GW use cost per crop and district')

#income_opt_j_d_e                   (j,  d)                                      Net income by crop and irrigation subdistrict
#income_opt_j_d_e                   (  j,d)..     income_opt_j_d_v               (  j,d)  =e= [revenue_opt_j_d_v(j,d) - prod_cost_opt_j_d_v(j,d) - sw_cost_opt_j_d_v(j,d) - gw_cost_opt_j_d_v(j,d)]
#                                                                                         -   [alpha_j_d_p(j,d) * Land_opt_j_d_v(j,d)] - [0.5 * gamma_j_d_p(j,d) * Land_opt_j_d_v(j,d)**2] ;
def income_opt_e(model, j,d):
  return (model.revenue_opt_j_d_v[j,d] - model.prod_cost_opt_j_d_v[j,d] - model.sw_cost_opt_j_d_v[j,d] - model.gw_cost_opt_j_d_v[j,d]) - (model.alpha_j_d_p[j,d] * model.land_opt_j_d_v[j,d]) - (0.5 * model.gamma_j_d_p[j,d] * model.land_opt_j_d_v[j,d] * model.land_opt_j_d_v[j,d]) == model.income_opt_j_d_v[j,d] 
model.income_opt_j_d_e = Constraint(model.j,model.d, rule=income_opt_e, doc='Net income by crop and irrigation subdistrict')

#income_opt_d_e                     (    d)                                      Net income in ach time step
#income_opt_d_e                     (    d)..     income_opt_d_v                 (    d)  =e= sum(j, income_opt_j_d_v(j,d)) ;
def tot_income_opt_e(model, d):
  return sum(model.income_opt_j_d_v[j,d] for j in model.j) == model.income_opt_d_v[d] 
model.income_opt_d_e = Constraint(model.d, rule=tot_income_opt_e, doc='Net income in ach time step')


## Define Objective and solve ##
#tot_income_opt_e                                                                Total income over all time steps  
#tot_income_opt_e                          ..     tot_income_opt_v                        =e= sum(d, income_opt_d_v(d)) ; 
#Model Pune1 /all/;
#Solve Pune1 using nlp maximizing tot_income_opt_v; 
def objective_rule(model):
  return sum(model.income_opt_d_v[d] for d in model.d)
model.objective = Objective(rule=objective_rule, sense=maximize, doc='Total income over all time steps ')

##Display everything in your model
#model.pprint()

##Display the constraints of the model 
#for con in model.component_map(Constraint).itervalues():
#    con.pprint()

## Creating and running the solver:
opt = SolverFactory("ipopt", solver_io='nl')
opt.options['max_iter']= 1000000 #number of iterations you wish
results = opt.solve(model,keepfiles=False, tee=True)
print(results.solver.termination_condition)
results.write()


##def pyomo_postprocess(options=None, instance=None, results=None):
model.land_opt_j_d_v.display()
model.objective.display()
 




ERROR: Rule failed when generating expression for constraint gw_cost_opt_j_d_e
    with index ('sorghum', '1'): NameError: name 'gw_use_opt_j_m_d_v' is not
    defined
ERROR: Constructing component 'gw_cost_opt_j_d_e' from data=None failed:
    NameError: name 'gw_use_opt_j_m_d_v' is not defined


NameError: name 'gw_use_opt_j_m_d_v' is not defined